# Prepare Malaya-Speech Dataset

We want our model able to understand Bahasa and local english slang (Manglish).

### Download data

Run command below to download all data,

```bash
wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-manglish.tar.gz
tar -xf semisupervised-manglish.tar.gz
wget https://cdn.commonvoice.mozilla.org/cv-corpus-5.1-2020-06-22/id.tar.gz
tar -zxf id.tar.gz
wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-malay.tar.gz
tar -xf semisupervised-malay.tar.gz
```

Total samples length,

1. Malay, ~93 hours.
2. Manglish, ~107 hours.
3. Indonesian, ~4 hours.

All these data are not really clean.

In [1]:
# !pip3 install malaya-speech -U --no-deps

### Read data

In [2]:
import pandas as pd
import malaya_speech
import malaya_speech.train as train
import json






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
df = pd.read_csv('cv-corpus-5.1-2020-06-22/id/validated.tsv', sep = '\t')
df = df[(df['sentence'].str.len() > 5) & (df['sentence'].str.count(' ') > 0)]
print(df.shape)

(7490, 10)


In [4]:
id_commonvoice = []
for i in range(len(df)):
    p = f"cv-corpus-5.1-2020-06-22/id/clips/{df['path'].iloc[i]}"
    id_commonvoice.append((p, df['sentence'].iloc[i]))

len(id_commonvoice)

7490

In [5]:
from glob import glob

malay = glob('semisupervised-malay/output-wav/*.wav')
manglish = glob('semisupervised-manglish/output-wav/*.wav')
len(malay), len(manglish)

(57895, 65277)

In [6]:
import unicodedata
import re


def preprocessing_text(string, reject = ',.!?#@\U0001f9da_，é)^🍪❤！' + '️'):
    for c in reject:
        string = string.replace(c, ' ')
        
    string = unicodedata.normalize('NFC', string.lower())
    return re.sub(r'[ ]+', ' ', string).strip()

In [7]:
from tqdm import tqdm

malays = []
for i in tqdm(malay):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        malays.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 57895/57895 [00:10<00:00, 5298.84it/s]


In [8]:
manglishs = []
for i in tqdm(manglish):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        manglishs.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 65277/65277 [00:12<00:00, 5207.67it/s]


In [9]:
audios = id_commonvoice + malays + manglishs
audios, texts = zip(*audios)

In [10]:
from pydub import AudioSegment
import numpy as np

def mp3_to_wav(file, sr = 16000):
    audio = AudioSegment.from_file(file)
    audio = audio.set_frame_rate(sr).set_channels(1)
    sample = np.array(audio.get_array_of_samples())
    return malaya_speech.astype.int_to_float(sample), sr

In [11]:
cleaned_texts = [preprocessing_text(t) for t in texts]

In [14]:
with open('malaya-speech-transcript.json', 'w') as fopen:
    json.dump(cleaned_texts, fopen)
    
with open('malaya-speech-transcript.txt', 'w') as fopen:
    fopen.write('\n'.join(cleaned_texts))

In [20]:
unique_chars = malaya_speech.char.generate_vocab(cleaned_texts)
unique_chars

['<PAD>',
 '<EOS>',
 ' ',
 'a',
 'e',
 'n',
 'i',
 't',
 'o',
 'u',
 's',
 'k',
 'r',
 'l',
 'h',
 'd',
 'm',
 'g',
 'y',
 'b',
 'p',
 'w',
 'c',
 'f',
 'j',
 'v',
 "'",
 '-',
 'z',
 '0',
 '1',
 'x',
 '2',
 'q',
 '*',
 '5',
 '3',
 '4',
 '6',
 '9',
 '8',
 '7',
 '%',
 '$',
 '"',
 '/',
 '&',
 ':',
 '+']

In [21]:
with open('malaya-speech-sst-vocab.json', 'w') as fopen:
    json.dump(unique_chars, fopen)

### Change into TFRecord

This is not necessary step, we recommend to use yield iterator to train the model, but we also can save our data into TFRecord to speed up data pipelines. To do that, we need to create a yield iterator.

In [22]:
def generator(maxlen = 16):
    for i in tqdm(range(len(audios))):
        try:
            if '.mp3' in audios[i]:
                wav_data, sr = mp3_to_wav(audios[i])
            else:
                wav_data, sr = malaya_speech.load(audios[i])
                
            if (len(wav_data) / sr) > maxlen or len(cleaned_texts[i]) < 5:
                print(f'skipped {audios[i]}')
                continue

            yield {
                'waveforms': wav_data.tolist(),
                'waveform_lens': [len(wav_data)],
                'targets': malaya_speech.char.encode(cleaned_texts[i], add_eos = False,
                                                     lookup = unique_chars),
                'raw_transcript': [cleaned_texts[i]],
            }
        except Exception as e:
            print(e)
            
generator = generator()

In [23]:
import os
import tensorflow as tf

os.system('rm bahasa-asr/data/*')
DATA_DIR = os.path.expanduser('bahasa-asr/data')
tf.gfile.MakeDirs(DATA_DIR)

#### Define shards

Like we defined below,

```python
shards = [{'split': 'train', 'shards': 999}, {'split': 'dev', 'shards': 1}]
```

In [24]:
shards = [{'split': 'train', 'shards': 999}, {'split': 'dev', 'shards': 1}]

#### Save to TFRecord

Just pass yield iterator to malaya_speech.train_prepare_dataset,

```python
def prepare_dataset(
    generator,
    data_dir: str,
    shards: List[Dict],
    prefix: str = 'dataset',
    shuffle: bool = True,
    already_shuffled: bool = False,
):
```

In [25]:
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')

  0%|          | 0/130662 [00:00<?, ?it/s]


INFO:tensorflow:Generating case 0.


  6%|▌         | 7513/130662 [11:00<1:12:46, 28.20it/s]

skipped semisupervised-malay/output-wav/Podcast-Bekpek---Berbasikal-Malaysia-Ke-Mekah(Ditahan-dan-dibawa-Kem-Askar-Pakistan-Semasa-Di-Gunung)-_bfH3pxuOoM.mp3-part-451.wav


  6%|▌         | 7750/130662 [11:03<30:43, 66.67it/s]  

...

100%|█████████▉| 130575/130662 [48:03<00:01, 60.61it/s]

skipped semisupervised-manglish/output-wav/Mamak-Sessions-Ep10---Datuk-Nicol-David-is-RETIRING!-n3AR3nQEc7Y.mp3-part-55.wav


100%|██████████| 130662/130662 [48:04<00:00, 45.30it/s]


INFO:tensorflow:Generated 128924 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`



INFO:tensorflow:Data shuffled.
